# Geodatenanalyse 1

## Übung 6: Statistisches Testen - Lösung

Nun wollen wir einige Hypothesen mit Hilfe von verschiedenen statistischen Tests an den Grundwasserdaten aus Karlsruhe testen. Lest dafür erneut den Datensatz "Data_GW_KA.csv" aus Übung 4 unter Verwendung eines geeigneten Variablennamen in Euer Jupyter Notebook ein. 

Entfernt dann mit Hilfe von `numpy.delete(array, column number, 0)` die Spalte mit den Eisenwerten, da diese NaN Werte enthält. 

In [1]:
# [1]
import numpy as np
data = np.genfromtxt(open("Data_GW_KA.csv", "rb"), delimiter=";", skip_header=1)

data = np.delete(data, 7, 0)
print(data)

[[3.90000e+01 9.70000e-01 1.44000e+01 6.08000e+02 6.98000e+00 1.00000e-01
  1.50000e+00 3.00000e+00 3.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00]
 [1.00000e+01 1.45000e+00 1.40000e+01 5.57670e+02 7.00000e+00 7.00000e-02
  7.00000e+00 2.00000e+00 2.00000e+00 2.00000e+00 1.00000e+00 2.00000e+00
  4.00000e+00 1.00000e+02 0.00000e+00]
 [1.58000e+01 1.07000e+00 1.25000e+01 5.92000e+02 7.06000e+00 1.00000e-01
  1.50000e+00 3.00000e+00 3.00000e+00 3.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00]
 [9.30000e+00 1.29000e+00 1.44000e+01 6.89000e+02 7.02000e+00 2.00000e-01
  1.50000e+00 3.00000e+00 3.00000e+00 4.00000e+00 1.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+02]
 [1.10000e+01 4.02000e+00 1.42000e+01 8.46000e+02 7.06000e+00 4.00000e-01
  7.33000e+00 3.00000e+00 3.00000e+00 1.00000e+00 1.00000e+00 3.00000e+00
  7.00000e+00 8.60000e+01 1.40000e+01]
 [1.40000e+01 2.18000e+00 1.28000e+01 3.48330e+02 7.29000e+00 2.3

Wir wollen in den Tests unter anderem die Daten im Hardtwald mit denen im Stadtgebiet vergleichen. Daher ist es sinnvoll den Datensatz aufzuspalten in einen Array mit den Werten aus dem Wald, und einen Array mit den Daten aus der Stadt. Schaut in der xlsx Datei nach der Spalte 'Flächennutzung' (1 = Stadt, 2 = Wald), und verwendet die richtigen Indices um die beiden Arrays zu erzeugen.  


In [2]:
# [2]
data_urban = data[0:30]
data_forest = data[31:39]

### Hypothese 1: "Die durchschnittliche Grundwassertemperatur im Wald beträgt 11°C" 

Zum Testen dieser Hypothese eignet sich ein zwei-seitiger t-Test. Die Nullhypothese H<sub>0</sub> haben wir bereits definiert, die alternative Hypothese lautet folglich "Die durchschnittliche Temperatur im Wald beträgt nicht 11°C". 

Der t-Test ist ein parametrischer Test, d.h. für einen gültigen Test müssen die Stichproben normal verteilt sein. Testet daher zuerst ob die Messwerte der Grundwassertemperatur diese Bedingung erfüllen. In `scipy.stats` gibt es die Funktion `shapiro()`, die einen Shapiro-Wilk-Test auf Normalität durchführt. Diese Funktion hat einen Input, den zu testenden Datensatz, und zwei Outputs, die Test-Statistik und den p-Wert. 

Führt den Shapiro-Wilk-Test für die Grundwassertemperatur durch und lasst euch den p-Wert anzeigen. Würdet Ihr für ein Signifikanzniveau von $\alpha$=0.01 annehmen, dass die Werte normalverteilt sind? 


In [3]:
# [3]
from scipy.stats import shapiro
stat, p = shapiro(data_forest[:,2])
print(p)

0.8608435988426208


Da der p-Wert deutlich größer ist als das Signifikanzniveau ($\alpha$=0.01), können wir die Nullhypothese, dass die Werte normalverteilt sind, annehmen. Also könnt Ihr mit dem t-test für eine Stichprobe fortfahren. 

In `scipy.stats` gibt es dafür die Funktion `ttest_1samp()`. Als Inputs benötigt diese Funktion den zu testenden Datensatz und den hypothetischen Mittelwert. Formuliert das Skript mit Hilfe einer `if else` Bedingung so, dass es Euch (zusätzlich zum p-Wert) bei einem resultierenden p-Wert von >0.01 "H0 annehmen", und ansonsten "H0 nicht annehmen" ausgibt. 


In [4]:
# [4]
from scipy import stats
stat, p = stats.ttest_1samp(data_forest[:,3], 11)
print('stat=%.4f, p=%.4f' % (stat, p))
if p > 0.01:
	print('H0 annehmen')
else:
	print('H0 nicht annehmen')




stat=23.2279, p=0.0000
H0 nicht annehmen


Wie habt Ihr Euch in Bezug auf die Nullhypothese von oben entschieden? Interessant ist natürlich auch zu sehen, wo der Mittelwerte der Probe eigentlich liegt. Berechnet diesen,und vergleicht ihn mit dem hypothetischen Wert. Was fällt auf? Woran könnte das liegen? 

Hinweis: Bestimmt neben dem Mittelwert auch die Anzahl an Messwerten in Eurem Datensatz. 

In [5]:
# [5]
import statistics
mean_GWT = statistics.mean(data_forest[:,3])
n = len(data_forest)
print(mean_GWT, n)

659.2857142857143 7


### Hypothese 2: "Brunnen in der Stadt haben eine niedrigere Sauerstoffsättigung als Brunnen im Hardtwald" 

Auch diese Nullhypothese können wir mit einem t-Test überprüfen. Da es sich um den Vergleich von zwei Datensätzen handelt, benötigen wir eine Funktion für einen two-sample t-Test. Muss dieser hier ein- oder zwei-seitig sein? 

Für einen gültigen two-sample t-Test müssen beiden Datensätze normalverteilt sein, und zusätzlich die gleiche Varianz haben. Testet zuerst die Hypothesen ob die Sauerstoffsättigung im Wald, sowie in der Stadt normalverteilt sind. 

In [6]:
# [6]
stat_f, p_f = shapiro(data_forest[:,1])
print(p_f)
stat_u, p_u = shapiro(data_urban[:,1])
print(p_u)

0.9151561856269836
0.1811668872833252


Zum Testen der Varianzen zweier Datensätze nimmt man häufig den F-Test. Dabei lautet die Null-Hypothese, dass die Varianzen gleich sind, und die alternative Hypothese, dass die Varianzen verschieden sind. 

Die Formel für die F-Statistik lautet: 

<img src="https://latex.codecogs.com/gif.latex?\hat{F}&space;=&space;\frac{s_{a}^{2}}{s_{b}^{2}}" title="\hat{F} = \frac{s_{a}^{2}}{s_{b}^{2}}" />

wobei die Datensätze *a* und *b* so gewählt werden müssen, das gilt: s<sub>a</sub><sup>2</sup> > s<sub>b</sub><sup>2</sup>. 

Berechnet zuerst die Varianzen der beiden Sauerstoff-Datensätze. Stellt für die Berechnung der F-Statistik ("F-Wert") sicher, dass das Skript die Größe der Varianzen korrekt berücksichtigt. 

In [7]:
# [7]
import statistics
var_urban = statistics.variance(data_urban[:,2])
var_forest = statistics.variance(data_forest[:,2])
if var_urban > var_forest:
    sa = var_urban
    sb = var_forest
else:
    sa = var_forest
    sb = var_urban
F = sa/sb
print (F)

128.99770114942518


Für das Testen der Hypothese wird dieser berechnete F-Wert nun mit dem kritischen F-Wert der Fisher-Verteilung verglichen, und der dazugehörige p-Wert berechnet. Dieser kann über `1 - scipy.stats.f.cdf()` berechnet werden. Die Funktion `scipy.stats.f.cdf()` benötigt als Inputs den F-Wert, und jeweils die Anzahl der Freiheitsgrade in beiden Datensätzen *a* und *b*. Da die Varianzen anhand der einzelnen Datenpunkte berechnet werden, entspricht die Anzahl an Freiheitsgraden jeweils `n-1`.

Lasst Euch den p-Wert anzeigen und vergleicht ihn mit dem Signifikanzniveau von 0.01. Würdet Ihr die Nullhypothese ("Beide Varianzen sind gleich") annehmen?


In [8]:
# [8]
import scipy
dfn = data_forest[:,2].size-1
dfd = data_urban[:,2].size-1
p = 1-scipy.stats.f.cdf(F, dfn, dfd)
print(p)

1.1102230246251565e-16


Da nun beide Voraussetzungen für den t-Test überprüft sind, könnt Ihr diesen nun anwenden. Eine Option dazu ist die Funktion `scipy.stats.ttest_ind()`. Als Input werden hier die beiden Mittelwerte der Datensätze *a* und *b*  benötigt, wobei mean(*a*) > mean(*b*). Als Output liefert die Funktion den T-Wert und den p-Wert. 

Rechnet die beiden Mittelwerte aus, stellt sicher, dass die Größenverhältnisse stimmen, und lasst Euch den T-Wert und den p-Wert ausgeben. Achtung: da die Hypothese einen ein-seitigen t-Test verlangt, müsst Ihr den resultierenden p-Wert noch durch 2 teilen. Sollte man die Nullhypothese hier annehmen?

In [9]:
# [9]
mean_oxy_forest = data_forest[:,2].mean()
mean_oxy_urban = data_urban[:,2].mean()

t_value, p = stats.ttest_ind(data_forest[:,2],data_urban[:,2])

p = p*0.5
print (t_value, p)

-5.34690250434321 2.8050214065297875e-06


Ähnlich wie für den F-Test gibt es auch beim t-Test die Möglichkeit den kritischen T-Wert zu berechnen und darüber den p-Wert zu bestimmen. Die Funktion dazu lautet `scipy.stats.t.ppf()`, und benötigt als Input den Wert "1-Signifikanzniveau/2" sowie die Summe der Freiheitsgrade. 

In [10]:
# [10]
df = dfn + dfd 
t_crit = scipy.stats.t.ppf(1-0.05/2, df)
print(t_crit)

2.0301079282503425


Wenn der berechnete T-Wert größer als der kritische T-Wert ist, sollte die Nullhypothese abgelehnt werden. Stimmt das Ergebnis hier mit dem oben überein? 

### Was tun wenn keine Normalverteilung der Werte vorliegt?

Viele reale Datensätze sind nicht normalverteilt. Das nicht-parametrische Äquivalent zum t-Test ist der Mann-Whitney U-Test. Die Nullhypothese zu diesem Test lautet, dass die beiden Datensätze die gleiche Verteilung haben. Die alternative Hypothese lautet, dass die Verteilungen nicht gleich sind. 

Die passende Funktion dafür ist `scipy.stats.mannwhitneyu()`, und benötigt als Input die beiden zu vergleichenden Datensätze. 

Identifiziert über eine `for-Schleife` in dem Datensatz nicht-normalverteilte Parameter, testet diese (`1. if-Bedingung`) auf eine gleiche Verteilung in Wald und Stadtbereich, und lasst euch diejenigen (`2. if-Bedingung`) anzeigen die eine gleiche Verteilung haben (also bei denen die Null-Hypothese angenommen werden kann, p-Wert > Signifikanzniveau). 

Tipp: Baut Euren Code Schritt für Schritt auf und testet zwischendurch immer wieder die Funktionalität. Erst die Schleife mit dem Shapiro-Test, dann die erste If-Bedingung, dann die zweite usw.  

In [11]:
# [11]

non_normal = []

for item in range(0, data.shape[1]-1): 
    #print(data[0, item])
    stat1, p1 = shapiro(data_forest[:,item])
    stat2, p2 = shapiro(data_urban[:,item])

    if p1 and p2 < 0.01:
        non_normal += [item]
        U_value, p_value = scipy.stats.mannwhitneyu(data_forest[:,item], data_urban[:,item])

        if p_value > 0.01:
            print (item, U_value, p_value)
        

5 44.0 0.01817155252427934
7 101.5 0.9014913145646711
8 116.0 0.6757058487030954
9 61.5 0.07002732167396186
11 136.0 0.2223315167433646
12 142.5 0.15119907218973086
13 126.0 0.4201182840484752


c:\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


Falls Ihr nun noch Zeit habt, könnt Ihr Euch zum Üben beispielsweise weitere Hypothesen zu anderen Parametern überlegen und die testen. 

## Ende